
## Translate body text (training) <br>

In the following, we translate the extracted body text of the articles from the training dataset. As previously mentioned, the limitations of Google Translate API require extra care in ordere to split the text (in order not to reach the character limit) and the set of documents (in order not to reach the document limit).

In [1]:
import pandas as pd
from googletrans import Translator
import time

In [2]:
train_df = pd.read_csv('train/_TRAIN_details_in_df.csv')

The limit over characters (max 15k characters according to the official documentation, but 5k in our tests) can be overcome by subdividing the text in a way that preserves basic text units. <br><br>

Our idea was subdividing the text into paragraphs (thus using `\n\n` as delimiter) until their length is shorter than 5000 characters. However, some articles presented some critical peculiarities (absence of paragraphs, absence of formatting, absence of punctuation) and that is a reason to observe that there is no obvious way to cut the text that will not raise some problems. <br><br>

Other attempts which we made are: cutting the text at the last punctuation symbol before the 5000 character limit, brutally cutting the text at 5000 characters and gluing the parts together after translation.

In [5]:
translator = Translator()

def splitTranslator(file,text,lang,delimiter='\n\n'):
    
    # break down long text into smaller chuck by splitting every time it sees the '\n\n' 
    # and keep it in body list 
    body_list =  text.split(delimiter)
    
    # iterate over each text chunks to translate
    translated_list = []
    
    flag_long_paragraph = False
    
    for s in body_list:
        
        # check if any chunk is still over 5000 char
        if len(s) > 5000:
            flag_long_paragraph = True
            print (file,'is still over limit:')
            continue
            
        # if the chunk is under 5000 char, then translate, and keep in the translated list
        elif len(s) == 0:
            continue
        else:
            try:
                translated = translator.translate(s)
                lang = translated.src
                translated = translated.text
                translated_list.append(translated)
            except Exception:
                print (file, 'cannot translate')
            time.sleep(0.5)
    
    # put all the translated text together and connect them using space
    translated_body = ' '.join(translated_list)
    return translated_body,flag_long_paragraph

## main translation task
count = 0

# we used start and end to translate in batches because of googletran's timeout
start = -1 
end = len(train_df)

long_paragraph_list = []
for index, row in train_df.iterrows():
    
    if index < start or index > end:
        continue
    
    print('--------------',index,'-----------------')
    id1 = str(row['pair_id']).split('_')[0]
    id2 = str(row['pair_id']).split('_')[1]
    body1 = str(row['text1'])
    lang1 = row['url1_lang']
    body2 = str(row['text2'])
    lang2 = row['url1_lang']
    
    if lang1 != 'en':
        print('translating text1 of language ', lang1,' of length ', len(body1),'...')
        if len(body1) > 5000:
            split_text1 = splitTranslator(id1,body1,lang1)
            translated1 = split_text1[0]
            if split_text1[1]:
                long_paragraph_list.append(id1)
        elif pd.isnull(body1):
            continue
        else:
            translated1 = translator.translate(body1)
            lang1 = translated1.src
            translated1 = translated1.text
    else:
        translated1 = body1.replace("\n", " ")
        
    if lang2 != 'en':
        print('translating text2 of language ', lang2,' of length ', len(body2),'...')
        if len(body2) > 5000:
            split_text2 = splitTranslator(id2,body2,lang2)
            translated2 = split_text2[0]
            if split_text2[1]:
                long_paragraph_list.append(id2)
        elif pd.isnull(body2):
            continue
        else:
            translated2 = translator.translate(body2)
            lang2 = translated2.src
            translated2 = translated2.text
    else:
        translated2 = body2.replace("\n", " ")
    
    train_df.loc[index, "translated_body1"] = translated1
    train_df.loc[index, "translated_body2"] = translated2
    
    time.sleep(0.5)
    count += 1
    
    if (count+1)%50 == 0:
        print (count+1, 'rows done.')
        

print("The number of articles that have been cut due to absence of paragraphs is ", len(long_paragraph_list))

-------------- 0 -----------------
-------------- 1 -----------------
-------------- 2 -----------------
-------------- 3 -----------------
-------------- 4 -----------------
-------------- 5 -----------------
-------------- 6 -----------------
-------------- 7 -----------------
-------------- 8 -----------------
-------------- 9 -----------------
-------------- 10 -----------------
-------------- 11 -----------------
-------------- 12 -----------------
-------------- 13 -----------------
-------------- 14 -----------------
-------------- 15 -----------------
-------------- 16 -----------------
-------------- 17 -----------------
-------------- 18 -----------------
-------------- 19 -----------------
-------------- 20 -----------------
-------------- 21 -----------------
-------------- 22 -----------------
-------------- 23 -----------------
-------------- 24 -----------------
-------------- 25 -----------------
-------------- 26 -----------------
-------------- 27 -----------------
--

-------------- 233 -----------------
-------------- 234 -----------------
-------------- 235 -----------------
-------------- 236 -----------------
-------------- 237 -----------------
-------------- 238 -----------------
-------------- 239 -----------------
-------------- 240 -----------------
-------------- 241 -----------------
-------------- 242 -----------------
-------------- 243 -----------------
-------------- 244 -----------------
-------------- 245 -----------------
-------------- 246 -----------------
-------------- 247 -----------------
-------------- 248 -----------------
250 rows done.
-------------- 249 -----------------
-------------- 250 -----------------
-------------- 251 -----------------
-------------- 252 -----------------
-------------- 253 -----------------
-------------- 254 -----------------
-------------- 255 -----------------
-------------- 256 -----------------
-------------- 257 -----------------
-------------- 258 -----------------
-------------- 259 ----

-------------- 573 -----------------
-------------- 574 -----------------
-------------- 575 -----------------
-------------- 576 -----------------
-------------- 577 -----------------
-------------- 578 -----------------
-------------- 579 -----------------
-------------- 580 -----------------
-------------- 581 -----------------
-------------- 582 -----------------
-------------- 583 -----------------
-------------- 584 -----------------
-------------- 585 -----------------
-------------- 586 -----------------
-------------- 587 -----------------
-------------- 588 -----------------
-------------- 589 -----------------
-------------- 590 -----------------
-------------- 591 -----------------
-------------- 592 -----------------
-------------- 593 -----------------
-------------- 594 -----------------
-------------- 595 -----------------
-------------- 596 -----------------
-------------- 597 -----------------
-------------- 598 -----------------
600 rows done.
-------------- 599 ----

-------------- 921 -----------------
-------------- 922 -----------------
-------------- 923 -----------------
-------------- 924 -----------------
-------------- 925 -----------------
-------------- 926 -----------------
-------------- 927 -----------------
-------------- 928 -----------------
-------------- 929 -----------------
-------------- 930 -----------------
-------------- 931 -----------------
-------------- 932 -----------------
-------------- 933 -----------------
-------------- 934 -----------------
-------------- 935 -----------------
-------------- 936 -----------------
-------------- 937 -----------------
-------------- 938 -----------------
-------------- 939 -----------------
-------------- 940 -----------------
-------------- 941 -----------------
-------------- 942 -----------------
-------------- 943 -----------------
-------------- 944 -----------------
-------------- 945 -----------------
-------------- 946 -----------------
-------------- 947 -----------------
-

-------------- 1243 -----------------
-------------- 1244 -----------------
-------------- 1245 -----------------
-------------- 1246 -----------------
-------------- 1247 -----------------
-------------- 1248 -----------------
1250 rows done.
-------------- 1249 -----------------
-------------- 1250 -----------------
-------------- 1251 -----------------
-------------- 1252 -----------------
-------------- 1253 -----------------
-------------- 1254 -----------------
-------------- 1255 -----------------
-------------- 1256 -----------------
-------------- 1257 -----------------
-------------- 1258 -----------------
-------------- 1259 -----------------
-------------- 1260 -----------------
-------------- 1261 -----------------
-------------- 1262 -----------------
-------------- 1263 -----------------
-------------- 1264 -----------------
-------------- 1265 -----------------
-------------- 1266 -----------------
-------------- 1267 -----------------
-------------- 1268 --------------

...

translating text1 of language  de  of length  3585 ...
translating text2 of language  de  of length  3602 ...
-------------- 4650 -----------------
translating text1 of language  de  of length  2726 ...
translating text2 of language  de  of length  3 ...
-------------- 4651 -----------------
translating text1 of language  de  of length  1975 ...
translating text2 of language  de  of length  2827 ...
-------------- 4652 -----------------
translating text1 of language  de  of length  1823 ...
translating text2 of language  de  of length  2497 ...
-------------- 4653 -----------------
translating text1 of language  de  of length  4089 ...
translating text2 of language  de  of length  3666 ...
-------------- 4654 -----------------
translating text1 of language  de  of length  931 ...
translating text2 of language  de  of length  680 ...
-------------- 4655 -----------------
translating text1 of language  de  of length  2654 ...
translating text2 of language  de  of length  2236 ...
-------

-------------- 4708 -----------------
translating text1 of language  de  of length  4997 ...
translating text2 of language  de  of length  3050 ...
-------------- 4709 -----------------
translating text1 of language  de  of length  4331 ...
translating text2 of language  de  of length  833 ...
-------------- 4710 -----------------
translating text1 of language  de  of length  2167 ...
translating text2 of language  de  of length  1784 ...
-------------- 4711 -----------------
translating text1 of language  de  of length  1583 ...
translating text2 of language  de  of length  789 ...
-------------- 4712 -----------------
translating text1 of language  de  of length  726 ...
translating text2 of language  de  of length  879 ...
-------------- 4713 -----------------
translating text1 of language  de  of length  4487 ...
translating text2 of language  de  of length  1301 ...
-------------- 4714 -----------------
translating text1 of language  de  of length  6055 ...
translating text2 of la

translating text1 of language  de  of length  7591 ...
translating text2 of language  de  of length  1779 ...
-------------- 4778 -----------------
translating text1 of language  de  of length  1593 ...
translating text2 of language  de  of length  3615 ...
-------------- 4779 -----------------
translating text1 of language  de  of length  13946 ...
translating text2 of language  de  of length  3562 ...
-------------- 4780 -----------------
translating text1 of language  de  of length  4263 ...
translating text2 of language  de  of length  2635 ...
-------------- 4781 -----------------
translating text1 of language  de  of length  6449 ...
translating text2 of language  de  of length  2171 ...
-------------- 4782 -----------------
translating text1 of language  de  of length  11609 ...
translating text2 of language  de  of length  2592 ...
-------------- 4783 -----------------
translating text1 of language  de  of length  1640 ...
translating text2 of language  de  of length  1457 ...


-------------- 4846 -----------------
translating text1 of language  ar  of length  1817 ...
translating text2 of language  ar  of length  616 ...
-------------- 4847 -----------------
translating text1 of language  ar  of length  1220 ...
translating text2 of language  ar  of length  749 ...
-------------- 4848 -----------------
translating text1 of language  ar  of length  4420 ...
translating text2 of language  ar  of length  2342 ...
4850 rows done.
-------------- 4849 -----------------
translating text1 of language  ar  of length  1279 ...
translating text2 of language  ar  of length  1591 ...
-------------- 4850 -----------------
translating text1 of language  ar  of length  2038 ...
translating text2 of language  ar  of length  900 ...
-------------- 4851 -----------------
translating text1 of language  ar  of length  700 ...
translating text2 of language  ar  of length  1564 ...
-------------- 4852 -----------------
translating text1 of language  ar  of length  376 ...
translat

-------------- 4911 -----------------
translating text1 of language  pl  of length  2116 ...
translating text2 of language  pl  of length  9999 ...
-------------- 4912 -----------------
translating text1 of language  pl  of length  1182 ...
translating text2 of language  pl  of length  2696 ...
-------------- 4913 -----------------
translating text1 of language  pl  of length  2142 ...
translating text2 of language  pl  of length  1805 ...
-------------- 4914 -----------------
translating text1 of language  pl  of length  2130 ...
translating text2 of language  pl  of length  1429 ...
-------------- 4915 -----------------
translating text1 of language  pl  of length  1454 ...
translating text2 of language  pl  of length  2521 ...
-------------- 4916 -----------------
translating text1 of language  pl  of length  603 ...
translating text2 of language  pl  of length  2638 ...
-------------- 4917 -----------------
translating text1 of language  pl  of length  1549 ...
translating text2 of

We observe that only 11 documents have been truncated due to absence of formatting or paragraphs.

In [25]:
# checking translated result
train_df.loc[2482, "translated_body1"]

"A woman who lost her husband to her own mother has said she will never fully forgive her for taking him away and then having his child.  Lauren Wall, who's now 34, and from Twickenham, south-west London married airport worker Paul White when she was just 19.  Her mum Julie, who is now 53, paid for a £15,000 wedding and grateful Lauren took her along on her two-week honeymoon to Devon.  Eight weeks later, husband Paul moved out and nine months later, her mother Julie gave birth to his child announcing they were together.  Lauren said: 'Paul always got on really well with mum. I never thought it strange though, as she was his mum-in-law and he was just being friendly.  Lauren Wall, from Twickenham, south-west London, who's now 34 got married to husband Paul when she was just 19. Just weeks later her husband moved out and was soon with Lauren's mum, Julie  'They'd laugh a lot together. I didn't think to be worried at all. Who would?  'I couldn't wait to settle in to marriage but the ink 

In [26]:
path = 'train/_TRAIN_text_translated.csv'
train_df.to_csv(path,index=False)